In [1]:
import os
import pickle

import numpy as np
import numpy.ma as ma

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from models import (
    NonHierarchicalBayesianModel
)


In [2]:
src = "/home/vishu/repos/rat-mapping-paper/reports/non-hierarchical/C_SMA_LAR/non_hierarchical_bayesian_model/inference.pkl"

with open(src, "rb") as f:
    df, encoder_dict, model, posterior_samples, = pickle.load(f)



An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
named_params = [site.a, site.b, site.L, site.ell, site.H]

x = np.arange(0, 500, .5)
x = x[*([None] * posterior_samples[site.a].ndim), :]
x.shape


(1, 1, 1, 1, 1, 1000)

In [4]:
posterior_samples = {
    u: v.reshape(4, -1, *v.shape[1:]) for u, v in posterior_samples.items() if u in named_params
}
posterior_samples = {
    u: v[:, -100:, ...] for u, v in posterior_samples.items()
}
posterior_samples = {
    u: v.reshape(-1, *v.shape[2:]) for u, v in posterior_samples.items()
}


In [5]:
y = F.rectified_logistic(
    x,
    *(posterior_samples[named_param][..., None] for named_param in named_params)
)
y = np.array(y)
y.shape


(400, 8, 70, 3, 6, 1000)

In [6]:
L = posterior_samples[site.L]
L.shape


(400, 8, 70, 3, 6)

In [7]:
y = y - L[..., None]
y.shape


(400, 8, 70, 3, 6, 1000)

In [8]:
subjects = sorted(df[model.features[0]].unique())
subjects_inv = [encoder_dict[model.features[0]].inverse_transform([subject]).item() for subject in subjects]

compound_positions = sorted(df[model.features[1]].unique())
compound_positions_inv = [encoder_dict[model.features[1]].inverse_transform([cpos]).item() for cpos in compound_positions]

compound_size = sorted(df[model.features[2]].unique())
compound_size_inv = [encoder_dict[model.features[2]].inverse_transform([csize]).item() for csize in compound_size]

combinations = df[model.features].apply(tuple, axis=1).unique().tolist()


In [11]:
posterior_samples = {
    u: ma.masked_array(v, mask=np.isnan(v)) for u, v in posterior_samples.items()
}
y = ma.masked_array(y, mask=np.isnan(y))


In [13]:
dest = os.path.join(model.build_dir, "processed_inference.pkl")

with open(dest, "wb") as f:
    pickle.dump(
        (df, encoder_dict, model, posterior_samples, y, subjects_inv, compound_positions_inv, compound_size_inv,),
        f
    )
